In [ ]:
import requests
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing as mp
from tqdm import tqdm
import scipy.signal.windows as win
from scipy.io import loadmat
from functools import partial

import speckit.flattop as ft
from speckit import compute_spectrum
from speckit.plotting import default_rc

pool = mp.Pool()
plt.rcParams.update(default_rc)

In [ ]:
# The following link should lead to a .mat datafile
# Contact the author at mikedovale@pm.me if the link is broken
url = 'https://www.dropbox.com/s/znmk8tfm0gv8a7b/GFO_Week52_2022.mat?dl=1'

# Download the file
response = requests.get(url)
with open('data.mat', 'wb') as f:
    f.write(response.content)

# Load the .mat file into a NumPy array
data = loadmat('data.mat')
x = np.array(data['RawPhase'][:,0])
print(x)

fs=9.664

In [ ]:
xlabel="Sample"
ylabel="Signal"
title="GRACE Follow On LRI range in meters - Calendar week 52, 2022"

fig, ax = plt.subplots(figsize=(8,3))
ax.plot(x, label="test", color="k")
ax.set_xlabel(xlabel)
ax.set_ylabel(ylabel)
ax.set_title(title)
fig.tight_layout()
fig.align_ylabels()
plt.show()

In [ ]:
# List of window functions available in scipy.signal.windows
windows_sc = {name: getattr(win, name) for name in dir(win) if callable(getattr(win, name))}
windows_sc

In [ ]:
# List of window functions available in spectools.flattop
windows_ft = {name: getattr(ft, name) for name in dir(ft) if callable(getattr(ft, name))}
windows_ft

In [ ]:
# List of window functions available in scipy.signal.windows
# windows = {name: getattr(win, name) for name in dir(win) if callable(getattr(win, name))}
# windows

windows = {
    'boxcar': win.boxcar,
    'triang': win.triang,
    # 'blackman': win.blackman,
    # 'hamming': win.hamming,
    'hann': win.hann,
    # 'bartlett': win.bartlett,
    # 'flattop': win.flattop,
    # 'parzen': win.parzen,
    # 'bohman': win.bohman,
    # 'blackmanharris': win.blackmanharris,
    'nuttall': partial(win.nuttall, sym=False),
    # 'barthann': win.barthann,
    'kaiser': win.kaiser,
    # 'tukey': win.tukey
    # "SFT3F": ft.SFT3F,
    # "SFT3M": ft.SFT3M,
    # "FTNI": ft.FTNI,
    # "SFT4F": ft.SFT4F,
    # "SFT5F": ft.SFT5F,
    # "SFT4M": ft.SFT4M,
    # "FTHP": ft.FTHP,
    "HFT70": ft.HFT70,
    # "FTSRS": ft.FTSR,
    # "SFT5M": ft.SFT5M,
    # "HFT90D": ft.HFT90D,
    # "HFT95": ft.HFT95,
    # "HFT116D": ft.HFT116D,
    # "HFT144D": ft.HFT144D,
    # "HFT169D": ft.HFT169D,
    # "HFT196D": ft.HFT196D,
    # "HFT223D": ft.HFT223D,
    # "HFT248D": ft.HFT248D
    }
windows

In [ ]:
res = {} # Dictionary to store results

In [ ]:
olap = 0.75
Jdes = 1000
Kdes = 100
force_target_nf = False
scheduler='lpsd'

for win_str, win_fun in tqdm(windows.items()):
    print(f"Computing spectrum with {win_str} window")
    res[win_str] = compute_spectrum(x, 
                            fs=fs, 
                            win=win_fun, 
                            olap=olap, 
                            Jdes=Jdes, 
                            Kdes=Kdes, 
                            force_target_nf=force_target_nf, 
                            scheduler=scheduler, 
                            pool=pool)
print("Done!")

In [ ]:
fig, ax = plt.subplots()

for win_str, psd in res.items():
    ax.loglog(psd.f, psd.asd, label=win_str)
ax.set_xlim([psd.f[0],psd.f[-1]])
ax.set_xlabel("Fourier frequency (Hz)")
ax.set_ylabel(r"ASD $\rm (A/Hz^{1/2})$")
ax.set_title("Comparison of different window functions in a high dynamic range signal")
ax.legend()
plt.show()